## We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

In [1]:
import json
import requests
response = requests.get('https://data.police.uk/api/crimes-street/all-crime?lat=52.63902&lng=-1.131321&date=2017-11')
all_crimes = response.json()
len(all_crimes)

1334

##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

In [11]:
def make_api_request(lat,lng,date=None):
    if date:
        url = 'https://data.police.uk/api/crimes-street/all-crime?lat=' + str(lat) + '&lng=' + str(lng) + '&date=' + date 
    else :
        url = 'https://data.police.uk/api/crimes-street/all-crime?lat=' + str(lat) + '&lng=' + str(lng)
    response = requests.get(url)
    return response.json()
    
# response = requests.get('https://data.police.uk/api/crimes-street/all-crime?lat=52.629729&lng=-1.131592&date=2017-01')
crimes = make_api_request(52.63902,-1.131321,'2017-11')

In [12]:
crimes[:3]

[{'category': 'anti-social-behaviour',
  'location_type': 'Force',
  'location': {'latitude': '52.635184',
   'street': {'id': 883410, 'name': 'On or near Shopping Area'},
   'longitude': '-1.135455'},
  'context': '',
  'outcome_status': None,
  'persistent_id': '',
  'id': 61222401,
  'location_subtype': '',
  'month': '2017-11'},
 {'category': 'anti-social-behaviour',
  'location_type': 'Force',
  'location': {'latitude': '52.637674',
   'street': {'id': 883469, 'name': 'On or near Tudor Road'},
   'longitude': '-1.147819'},
  'context': '',
  'outcome_status': None,
  'persistent_id': '',
  'id': 61223188,
  'location_subtype': '',
  'month': '2017-11'},
 {'category': 'anti-social-behaviour',
  'location_type': 'Force',
  'location': {'latitude': '52.629501',
   'street': {'id': 883363, 'name': 'On or near Grange Lane'},
   'longitude': '-1.136424'},
  'context': '',
  'outcome_status': None,
  'persistent_id': '',
  'id': 61222667,
  'location_subtype': '',
  'month': '2017-11'}]

##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)



**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Edinburgh Castle in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

In [47]:
buckingham = (51.5017861,-0.1432319)
edinburgh = (55.948811,-3.197982)
manchester = (53.480161,-2.245163)

In [48]:
def categories_of_crime(lat,lng,date = None):
    crimes = make_api_request(lat,lng,date)
    category_dict = {}
    for crime in crimes:
        if crime['category'] in category_dict:
            category_dict[crime['category']] +=1
        else:
            category_dict[crime['category']] = 1
    return category_dict

In [41]:
buck_categories = categories_of_crime(buckingham[0],buckingham[1])
print('Buckingham Palace crime categories:',buck_categories)

Buckingham Palace crime categories: {'anti-social-behaviour': 608, 'bicycle-theft': 65, 'burglary': 138, 'criminal-damage-arson': 82, 'drugs': 74, 'other-theft': 810, 'possession-of-weapons': 26, 'public-order': 174, 'robbery': 141, 'shoplifting': 329, 'theft-from-the-person': 586, 'vehicle-crime': 102, 'violent-crime': 553, 'other-crime': 22}


In [42]:
edin_categories = categories_of_crime(edinburgh[0],edinburgh[1])
print('Edinburgh Castle crime categories',edin_categories)

Edinburgh Castle crime categories {'criminal-damage-arson': 1, 'drugs': 1, 'other-theft': 9, 'possession-of-weapons': 1, 'public-order': 1, 'shoplifting': 4, 'violent-crime': 7, 'other-crime': 2}


In [46]:
manch_categories = categories_of_crime(manchester[0],manchester[1])
print(manch_categories)

{'anti-social-behaviour': 422, 'bicycle-theft': 86, 'burglary': 95, 'criminal-damage-arson': 132, 'drugs': 42, 'other-theft': 357, 'possession-of-weapons': 25, 'public-order': 289, 'robbery': 134, 'shoplifting': 159, 'theft-from-the-person': 321, 'vehicle-crime': 284, 'violent-crime': 607, 'other-crime': 25}


##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

In [56]:
from collections import Counter

def find_outcome_statuses(lat,lng,date=None):
    outcome_status = []
    crimes = make_api_request(lat,lng,date)
    for crime in crimes:
        if crime['outcome_status']:
            outcome_status.append(crime['outcome_status']['category'])
        else:
            outcome_status.append('Not Resolved')
    d = Counter(outcome_status)    
    return d
    

In [57]:
buck_outcomes = find_outcome_statuses(buckingham[0],buckingham[1])
print(buck_outcomes)

Counter({'Under investigation': 2730, 'Not Resolved': 608, 'Investigation complete; no suspect identified': 201, 'Awaiting court outcome': 85, 'Local resolution': 66, 'Offender given a caution': 15, 'Offender given penalty notice': 3, 'Offender given a drugs possession warning': 2})


In [59]:
manch_outcomes = find_outcome_statuses(manchester[0],manchester[1],'2017-10')
print(manch_outcomes)

Counter({'Investigation complete; no suspect identified': 1834, 'Not Resolved': 445, 'Unable to prosecute suspect': 376, 'Status update unavailable': 266, 'Further investigation is not in the public interest': 92, 'Court result unavailable': 72, 'Local resolution': 44, 'Awaiting court outcome': 19, 'Offender sent to prison': 17, 'Offender given community sentence': 14, 'Formal action is not in the public interest': 13, 'Offender given a drugs possession warning': 11, 'Offender given a caution': 9, 'Offender given suspended prison sentence': 8, 'Offender fined': 4, 'Offender given penalty notice': 2, 'Defendant sent to Crown Court': 2, 'Defendant found not guilty': 2, 'Court case unable to proceed': 2, 'Offender given conditional discharge': 1, 'Offender given absolute discharge': 1})


##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

In [60]:
def month_highest_crimes(lat,lng,year):
    month_crimes = {}
    for month in range(1,13):
        month_crimes[str(month)] = len(make_api_request(lat,lng,year+'-' +str(month)))
    return max(month_crimes, key=month_crimes.get)

In [61]:
manch_highest_month = month_highest_crimes(manchester[0],manchester[1],'2016')

In [62]:
manch_highest_month

'12'

## Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?